<p style="text-align: center">
    <img src="../../assets/images/untref-logo-negro.svg" />
</p>

<h3 style="text-align: center">Estructuras de Datos</h3>

<h2 style="text-align: center">Clase 8: Recuperación de la información semi-estructurada ~ XML y XPath</h2>

### XML: Extensible Markup Language

* Es un **lenguaje de etiquetas** y también un **formato de archivo**.
* Se usa para guardar, transferir y reconstruir información arbitraria.
* Es una especificación definida por la [World Wide Web Consortium (W3C)](https://www.w3.org/).
* Define una serie de reglas para codificar documentos en un formato legible tanto por computadoras como por humanos.

Un documento XML es un árbol ordenado, donde cada nodo se rotula con dos etiquetas, una de apertura y otra de cierre.

Por ejemplo el siguiente fragmento XML estructura un poema:

``` xml
<poema fecha="Abril de 1915" lugar="Granada">
	<titulo>Alba</titulo>
	<verso>Mi corazón oprimido</verso>
	<verso>siente junto a la alborada</verso>
	<verso>el dolor de sus amores</verso>
	<verso>y el sueño de las distancias. </verso>
</poema>
```

Existen varios sitios en internet que permiten visualizar el árbol asociado. Por ejemplo [https://codebeautify.org/xmlviewer](https://codebeautify.org/xmlviewer?input=%3Cpoema%20fecha=%22Abril%20de%201915%22%20lugar=%22Granada%22%3E%3Ctitulo%3EAlba%3C/titulo%3E%3Cverso%3EMi%20coraz%C3%B3n%20oprimido%3C/verso%3E%3Cverso%3Esiente%20junto%20a%20la%20alborada%3C/verso%3E%3Cverso%3Eel%20dolor%20de%20sus%20amores%3C/verso%3E%3Cverso%3Ey%20el%20sue%C3%B1o%20de%20las%20distancias.%20%3C/verso%3E%3C/poema%3E)

#### Elementos clave

##### Marcado vs. Contenido

* El texto en un archivo XML se separan entre _marcado_ y _contenido_.
* Generalmente, todo lo que es _marcado_ está encerrado entre `<` y `>`, o comienzan con `&` y terminan con `;`
* Todo lo que no es _marcado_ es _contenido_.
* A excepción de la sección `CDATA`, donde se encierra _contenido_ entre `<![CDATA[` y `]]>`.
* El espacio en blanco entre _marcado_ es ignorado.

##### Etiqueta

Una _etiqueta_ es una estructura de marcado que comienza con `<` y termina con `>`.

Hay 3 tipos:

* Etiqueta de inicio: `<etiqueta>`
* Etiqueta de cierre: `</etiqueta>`
* Etiqueta sin elementos: `<etiqueta />`

##### Elemento

Un _elemento_ es un componente lógico de un documento que comienza con una etiqueta de inicio y termina con una etiqueta de cierre correspondiente o que consta únicamente de una etiqueta sin elementos.

Lo que esté entre las etiquetas de inicio y cierre, son el contenido del _elemento_ y puede estar compuesto de otros elementos.

Los elementos dentro de elementos, se denominan los _hijos_ del elemento _padre_.

Un ejemplo es:

```xml
<greeting>Hello, world!</greeting>
```

Otro es:

```xml
<line-break />
```

##### Atributo

Un _atributo_ es una estructura de marcado que consiste en un par nombre-valor que existe dentro de una etiqueta de inicio o una etiqueta de elemento vacío.

Un ejemplo es:

```xml
<img src="messi.jpg" alt="Messi" />
```

donde los nombres de los atributos son `src` y `alt`, y sus valores son `messi.jpg` y `Messi` respectivamente.

Otro ejemplo es:

```xml
<step number="3">Connect A to B.</step>
```

donde el nombre del atributo es `number` y su valor es `3`.

* Un atributo solo puede tener un único valor.
* Cada atributo puede aparecer como máximo una vez en cada elemento.
* Si se necesita que un atributo almacene una lista, esos valores se tienen que serializar.

##### Caracteres especiales

* XML soporta todos los caracteres Unicode (`UTF-8`), excepto el caracter nulo (`0x0`).
* Los caracteres `<` y `&` son marcadores de sintaxis claves.
    * Esos caracteres no puede aparecer nunca como contenido, excepto en bloques `CDATA`.
    * Para escapar estos caracteres debemos utilizar las siguientes representaciones:
        * `&lt;` representa `<`
        * `&gt;` representa `>`
        * `&amp;` representa `&`
        * `&apos;` representa `'`
        * `&quot;` representa `"`

##### Comentarios

```xml
<!--
Commentarios 
-->

#### Ejemplos de uso de documentos XML

* Catálogos de bibliotecas
* Información sobre patentes
* Procesadores de texto (`docx`, `odt`, `xlsx`, `ods`, `pptx`, `odp`, etc.)
* Noticias (RSS)

##### RSS

En general las agencias de noticias y los diarios acostumbran publicar resumenes de noticias en formato [RSS](https://es.wikipedia.org/wiki/RSS), un formato XML específico que se utiliza para difundir información actualizada frecuentemente a usuarios que se han suscrito a la fuente de contenidos.

Por ejemplo podemos consultar las últimas noticias del blog de Python:

https://blog.python.org/feeds/posts/default?alt=rss

In [ ]:
from urllib.request import urlopen

from lxml import etree

response = urlopen("https://blog.python.org/feeds/posts/default?alt=rss")
response_body = response.read()
tree = etree.fromstring(response_body)
formatted_xml = etree.tostring(tree, pretty_print=True)
print(formatted_xml.decode())

### XPath: XML Path Language

* Estándar para navegar documentos XML.
* Permite describir caminos en un árbol XML.
* Evaluar una expresión XPath es buscar si hay **elementos** en el documento que se ajustan al recorrido definido en la expresión.
* El resultado de la evaluación son todos los **elementos** que se ajustan a la expresión.
* Para poder evaluar una expresión XPath, el documento debe estar bien formado.

Vamos a usar el siguiente documento para los ejemplos:

``` xml
<?xml version="1.0" encoding="UTF-8"?>
<biblioteca>
  <libro>
    <titulo>La vida está en otra parte</titulo>
    <autor>Milan Kundera</autor>
    <fechaPublicacion año="1973" />
    <precio>305.50</precio>
  </libro>
  <revista>
    <titulo>Computer Gaming World</titulo>
    <editorial>Golden Empire Publications</editorial>
    <fechaPublicacion año="1981" />
    <precio>669.99</precio>
  </revista>
  <libro>
    <titulo>Pantaleón y las visitadoras</titulo>
    <autor fechaNacimiento="28/03/1936">Mario Vargas Llosa</autor>
    <fechaPublicacion año="1973" />
    <precio>214.48</precio>
  </libro>
  <libro>
    <titulo>Conversación en la catedral</titulo>
    <autor fechaNacimiento="28/03/1936">Mario Vargas Llosa</autor>
    <fechaPublicacion año="1969" />
    <precio>541.78</precio>
  </libro>
  <revista>
    <titulo>PC Users</titulo>
    <editorial>RedUsers</editorial>
    <fechaPublicacion año="2000" />
    <precio>220.50</precio>
  </revista>
</biblioteca>
```

Existen herramientas online para probar, por ejemplo https://codebeautify.org/Xpath-Tester

#### Nodos XPath

Hay 7 clases de nodos: **elemento**, **atributo**, **texto**, espacio de nombre, instrucción, comentario y documentación.

##### Nodo raíz

``` xml
<biblioteca>
```

##### Elemento

``` xml
<titulo>La vida está en otra parte</titulo> 
```

##### Atributo

``` xml
fechaNacimiento="28/03/1936"
```
##### Texto

```
Pantaleón y las visitadoras
```

### Primero, algunos detalles técnicos

Vamos a usar el paquete `lxml`, el cual debemos instalar utilizando `pip`, desde esta misma notebook, podemos ejecutar

**El paquete `lxml` implementa [XPath Version 1.0](https://www.w3.org/TR/xpath-10/)**

In [ ]:
%pip install lxml

Ahora, vamos a importar este módulo esta única vez.

In [ ]:
from lxml import etree

También vamos a crear una función que nos facilitará la impresión de los resultados que tengamos cuando ejecutemos los selectores de XPath.

In [ ]:
def print_xml(elements: list[etree.Element]) -> None:
    if len(elements) == 0:
        print("~ Sin resultados ~")
        return

    element_strings = [
        (e.strip() if isinstance(e, str) else etree.tostring(e, encoding="utf-8", pretty_print=True).decode().strip())
        for e in elements
    ]

    print("\n~~~\n".join(element_strings))

Por último, cargamos el documento XML que vamos a utilizar en todos los ejemplos.

In [ ]:
doc = etree.parse("biblioteca.xml")
doc

### Expresiones XPath para seleccionar nodos

| Expresión | Descripción |
| :- | :- |
| `/` | Si está al principio de la expresión, indica el nodo raíz, si no, indica "hijo"|
| `//` | Camino. Permite seleccionar nodos en un camino descendiente a partir de la posición actual |
| `.` | Nodo actual |
| `..` | Padre del nodo actual |
| `@nombre_atributo` | Atributo |

In [ ]:
doc.xpath("/biblioteca/libro/titulo")

In [ ]:
print_xml(doc.xpath("/biblioteca/libro/titulo"))

In [ ]:
print_xml(doc.xpath("//titulo"))

In [ ]:
print_xml(doc.xpath("//titulo/."))

In [ ]:
print_xml(doc.xpath("//titulo/.."))

### Predicados

Los predicados pueden ser usados para filtrar un conjunto de nodos en base a una condición dada.

Los predicados se escriben entre corchetes (`[`, `]`).

| Expresión | Resultado |
| :- | :- |
| `/biblioteca/libro[position() = 1]` | Selecciona libro en la posición 1, hijo de biblioteca |
| `/biblioteca/libro[1]` | La misma condición en su versión resumida |
| `/biblioteca/libro[last()]` | Selecciona el último nodo libro, hijo de biblioteca |
| `/biblioteca/libro[last() - 1]` | Selecciona el ante último nodo libro, hijo de biblioteca | 
| `/biblioteca/libro[position() < 3]` | Selecciona los dos primeros nodos libro, hijo de biblioteca |
| `//autor[@fechaNacimiento]` | Selecciona todos los nodos autor, que tengan un atributo fechaNacimiento |
| `//autor[not(@fechaNacimiento)]` | Selecciona todos los nodos autor, que **no** tengan un atributo fechaNacimiento |
| `//autor[@fechaNacimiento="28/03/1936"]` | Selecciona todos los nodos autor, que tengan un atributo fechaNacimiento con un valor específico |
| `/biblioteca//libro[precio < 350]` | Selecciona todos los nodos libros, que tengan como hijo directo un elemento precio con valor menor a 350 |
| `/biblioteca//libro[precio < 350]/titulo` | Selecciona todos los nodos titulo, hijos de nodos libros que tengan como hijo directo un elemento precio con valor menor a 350 |

In [ ]:
print_xml(doc.xpath("/biblioteca/libro[position() = 1]"))

In [ ]:
print_xml(doc.xpath("/biblioteca/libro[1]"))

In [ ]:
print_xml(doc.xpath("/biblioteca/libro[last()]"))

In [ ]:
print_xml(doc.xpath("/biblioteca/libro[last() - 1]"))

In [ ]:
print_xml(doc.xpath("/biblioteca/libro[position() < 3]"))

In [ ]:
print_xml(doc.xpath("//autor[@fechaNacimiento]"))

In [ ]:
print_xml(doc.xpath("//autor[not(@fechaNacimiento)]"))

In [ ]:
print_xml(doc.xpath('//autor[@fechaNacimiento="28/03/1936"]'))

In [ ]:
print_xml(doc.xpath("/biblioteca//libro[precio < 350]"))

In [ ]:
print_xml(doc.xpath("/biblioteca//libro/precio[. < 350]"))

In [ ]:
print_xml(doc.xpath("/biblioteca//libro[precio < 350]/titulo"))

### Selectores y comodines

| Expresión | Resultado |
| :- | :- |
| `/biblioteca/*` | Selecciona todos los elementos hijos directos de biblioteca |
| `/biblioteca//*` | Selecciona todos los elementos descendientes de biblioteca |
| `//autor[@*]` | Selecciona todos los elementos autor que tengan algún atributo |
| `node()` | Selecciona todos los elementos autor que tengan algún atributo |
| `//titulo/text()` | Selecciona el texto (no el nodo completo) de los títulos |

In [ ]:
print_xml(doc.xpath("/biblioteca/*"))

In [ ]:
print_xml(doc.xpath("/biblioteca//*"))

In [ ]:
print_xml(doc.xpath("//autor[@*]"))

In [ ]:
print_xml(doc.xpath("//*[@*]"))

In [ ]:
print_xml(doc.xpath("node()"))

In [ ]:
doc.xpath("/biblioteca/node()")

In [ ]:
print_xml(doc.xpath("//titulo/text()"))

### Selección de varios caminos

El operador `|` es el operador de unión.

| Expresión | Resultado |
| :- | :- |
| `//libro/titulo \| //libro/precio` | Selecciona todos los nodos titulo y precio hijos de libro |

In [ ]:
print_xml(doc.xpath("//libro/titulo | //libro/precio"))

In [ ]:
print_xml(doc.xpath("//libro/precio | //libro/titulo"))

### Comparaciones

| Operador | Descripción | Ejemplo |
| :- | :- | :- |
| `=` | Igualdad | `precio = 541.78` |
| `!=` | Distinto | `precio != 541.78` |
| `<` | Menor estricto | `precio < 500` |
| `<=` | Menor o igual | `precio <= 541.78` |
| `>` | Mayor estricto | `precio > 500` |
| `<=` | Mayor o igual | `precio >= 541.78` |

- Si **ambos objetos** que se van a comparar son **conjuntos de nodos**, la comparación será verdadera si y solo si hay un nodo en el primer conjunto de nodos y un nodo en el segundo conjunto de nodos de modo que el resultado de realizar la comparación en los valores de cadena de los dos nodos sea verdadero.

- Si un objeto que se va a comparar es un **conjunto de nodos** y el otro es un **número**, la comparación será verdadera si y solo si hay un nodo en el conjunto de nodos de modo que el resultado de realizar la comparación en el número que se va a comparar y en el resultado de convertir el valor de cadena de ese nodo en un número utilizando la función number sea verdadero.

- Si un objeto que se va a comparar es un **conjunto de nodos** y el otro es una **cadena**, la comparación será verdadera si y solo si hay un nodo en el conjunto de nodos de modo que el resultado de realizar la comparación en el valor de cadena del nodo y la otra cadena sea verdadero.

- Si un objeto a comparar es un **conjunto de nodos** y el otro es un **booleano**, entonces la comparación será verdadera si y solo si el resultado de realizar la comparación en el booleano y en el resultado de convertir el conjunto de nodos en un booleano usando la función booleana es verdadero.

- Cuando **ninguno de los objetos** a comparar es un **conjunto de nodos** y el operador es `=` o `!=`, entonces los objetos se comparan convirtiéndolos a un tipo común de la siguiente manera y luego comparándolos.

    - Si al menos un objeto a comparar es un **booleano**, entonces cada objeto a comparar se convierte en un booleano como si se aplicara la función booleana.
    - De lo contrario, si al menos un objeto a comparar es un **número**, entonces cada objeto a comparar se convierte en un número como si se aplicara la función numérica.
    - De lo contrario, **ambos objetos a comparar se convierten en cadenas** como si se aplicara la función de cadena.
    - La comparación `=` será verdadera si y solo si los objetos son iguales.
    - La comparación `!=` será verdadera si y solo si los objetos no son iguales.
    - Los números se comparan para comprobar su igualdad según _IEEE 754_. Dos valores booleanos son iguales si ambos son verdaderos o ambos son falsos.
    - Dos cadenas son iguales si y solo si consisten en la misma secuencia de caracteres UCS.

    > **NOTA**: Si `$x` está vinculado a un conjunto de nodos, entonces `$x="foo"` no significa lo mismo que `not($x!="foo")`: el primero es verdadero si y solo si algún nodo en `$x` tiene el valor de cadena foo; el último es verdadero si y solo si todos los nodos en `$x` tienen el valor de cadena foo.

- Cuando **ninguno de los objetos** a comparar es un **conjunto de nodos** y el operador es `<=`, `<`, `>=` o `>`, entonces los objetos se comparan convirtiendo ambos objetos en números y comparando los números según _IEEE 754_.

    - La comparación `<` será verdadera si y solo si el primer número es menor que el segundo número.
    - La comparación `<=` será verdadera si y solo si el primer número es menor o igual que el segundo número.
    - La comparación `>` será verdadera si y solo si el primer número es mayor que el segundo número.
    - La comparación `>=` será verdadera si y solo si el primer número es mayor o igual que el segundo número.

Fuente: [W3C > XML Path Language (XPath) Version 1.0 > 3.4 Booleans](https://www.w3.org/TR/xpath-10/#booleans)

In [ ]:
print_xml(doc.xpath("//autor[. = 'Milan Kundera']"))

In [ ]:
print_xml(doc.xpath("//autor[. != 'Milan Kundera']"))

In [ ]:
print_xml(doc.xpath("(//libro | //revista)[precio > 300]"))

### Operadores

| Operador | Descripción | Ejemplo |
| :- | :- | :- |
| `+` | Suma | `6 + 4` |
| `-` | Sustracción | `6 - 4` |
| `*` | Multiplicación | `6 * 4` |
| `div` | División | `8 div 4` |
| `or` | Disyunción | `precio = 541.78 or precio = 214.48` |
| `and` | Conjunción | `precio > 300 and precio <= 541.78` |
| `mod` | Módulo (resto de la división entera) | `5 mod 2` |

In [ ]:
print_xml(doc.xpath("//libro[position() mod 2 = 1]"))

In [ ]:
print_xml(doc.xpath("//libro[position() mod 2 = 1 and autor[@fechaNacimiento]]"))

In [ ]:
print_xml(doc.xpath("//libro[(position() mod 2 = 1 and autor[@fechaNacimiento]) or precio < 300]"))

In [ ]:
print_xml(doc.xpath("//libro[precio < 300 or (position() mod 2 = 1 and autor[@fechaNacimiento])]"))

### XML y Python

In [ ]:
from lxml import etree

tree = etree.parse("biblioteca.xml")  # archivo.xml contiene el ejemplo de la biblioteca
tree

In [ ]:
root = tree.getroot()  # Devuelve la raíz del árbol XML
root

In [ ]:
len(root)  # los elementos del árbol son listas

In [ ]:
for child in root:
    print(child)

In [ ]:
for child in root:
    print(child.tag)  # con `.tag` se accede a la etiqueta

In [ ]:
# Todos los autores

b1 = tree.xpath("//autor")
for autor in b1:
    print(autor.text)

In [ ]:
# Todos los libros que cuestan entre 300 y 541.78 inclusive

b2 = tree.xpath("//libro[precio > 300 and precio <= 541.78]/titulo")
for libro in b2:
    print(libro.text)

In [ ]:
# Todos los libros que cuestan entre 300 y 541.78 inclusive
b3 = tree.xpath("//libro[precio > 300 and precio <= 541.78]/titulo/text()")
for libro in b3:
    print(libro)

---

### Trivia

```xml
<?xml version="1.0" encoding="UTF-8"?>
<geografia>
  <continentes>
    <continente nombre="Europa">
      <pais>España</pais>
      <pais>Francia</pais>
      <pais>Suiza</pais>
    </continente>
    <continente nombre="América">
      <pais>Argentina</pais>
      <pais>Jamaica</pais>
      <pais>Uruguay</pais>
    </continente>
  </continentes>
  <paises>
    <pais nombre="España">
      <sistema>monarquía</sistema>
      <superficie>504645</superficie>
      <moneda>euro</moneda>
      <moneda antigua="antigua">peseta</moneda>
      <idioma>español</idioma>
    </pais>
    <pais nombre="Francia">
      <sistema>república</sistema>
      <superficie>675417</superficie>
      <moneda>euro</moneda>
      <moneda antigua="antigua">franco francés</moneda>
      <idioma>francés</idioma>
    </pais>
    <pais nombre="Suiza">
      <sistema>república</sistema>
      <superficie>41290</superficie>
      <moneda>franco suizo</moneda>
      <idioma>francés</idioma>
    </pais>
    <pais nombre="Uruguay">
      <sistema>república</sistema>
      <superficie>176215</superficie>
      <moneda>peso uruguayo</moneda>
      <idioma>español</idioma>
    </pais>
    <pais nombre="Argentina">
      <sistema>república</sistema>
      <superficie>2780400</superficie>
      <moneda>peso argentino</moneda>
      <idioma>español</idioma>
    </pais>
    <pais nombre="Jamaica">
      <sistema>monarquía</sistema>
      <superficie>10991</superficie>
      <moneda>dólar jamaicano</moneda>
      <idioma>inglés</idioma>
    </pais>
  </paises>
  <rios>
    <rio>
      <nombre>Uruguay</nombre>
      <pais>Argentina</pais>
      <pais>Uruguay</pais>
    </rio>
  </rios>
</geografia>
```

http://xpather.com/4qdQv7Dc

In [ ]:
from lxml import etree

doc = etree.parse("geografia.xml")

1. Todos los países que tienen como moneda al euro
    - https://app.sli.do/event/sbWVuCZzRjU4NdVXGM3YSr/embed/polls/63cd6017-8126-4842-b31f-ba4c6d360c51

In [ ]:
print_xml(doc.xpath("/"))

2. Los nombres de los ríos europeos (ríos que atraviesan países en el continente europeo)
    - https://app.sli.do/event/sbWVuCZzRjU4NdVXGM3YSr/embed/polls/e11edd47-929c-44fd-9bab-fb0ef36908fc

In [ ]:
print_xml(doc.xpath("/"))

3. Superficie de los países por donde pasa el rio Uruguay
    - https://app.sli.do/event/sbWVuCZzRjU4NdVXGM3YSr/embed/polls/a7c41b39-d442-488a-ba25-c9c8187a6c01

In [ ]:
print_xml(doc.xpath("/"))

```xml
<?xml version="1.0" encoding="UTF-8"?>
<Materias>
  <Departamento Codigo="CS">
    <Titulo>Computer Science</Titulo>
    <Director>
      <Profesor>
        <Nombre>Jennifer</Nombre>
        <Apellido>Widom</Apellido>
      </Profesor>
    </Director>
    <Materia Numero="CS106A" Vacantes="1070">
      <Titulo>Programming Methodology</Titulo>
      <Descripcion>Introduction to the engineering of computer applications emphasizing modern software engineering principles.</Descripcion>
      <Docentes>
        <Ayudante>
          <Nombre>Jerry</Nombre>
          <Segundo_Nombre>R.</Segundo_Nombre>
          <Apellido>Cain</Apellido>
        </Ayudante>
        <Profesor>
          <Nombre>Eric</Nombre>
          <Apellido>Roberts</Apellido>
        </Profesor>
        <Profesor>
          <Nombre>Mehran</Nombre>
          <Apellido>Sahami</Apellido>
        </Profesor>
      </Docentes>
    </Materia>
    <Materia Numero="CS106B" Vacantes="620">
      <Titulo>Programming Abstractions</Titulo>
      <Descripcion>Abstraction and its relation to programming.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>Eric</Nombre>
          <Apellido>Roberts</Apellido>
        </Profesor>
        <Ayudante>
          <Nombre>Jerry</Nombre>
          <Segundo_Nombre>R.</Segundo_Nombre>
          <Apellido>Cain</Apellido>
        </Ayudante>
      </Docentes>
      <Correlativas>
        <Corre>CS106A</Corre>
      </Correlativas>
    </Materia>
    <Materia Numero="CS107" Vacantes="500">
      <Titulo>Computer Organization and Systems</Titulo>
      <Descripcion>Introduction to the fundamental concepts of computer systems.</Descripcion>
      <Docentes>
        <Ayudante>
          <Nombre>Julie</Nombre>
          <Apellido>Zelenski</Apellido>
        </Ayudante>
      </Docentes>
      <Correlativas>
        <Corre>CS106B</Corre>
      </Correlativas>
    </Materia>
    <Materia Numero="CS109" Vacantes="280">
      <Titulo>Introduction to Probability for Computer Scientists</Titulo>
      <Docentes>
        <Profesor>
          <Nombre>Mehran</Nombre>
          <Apellido>Sahami</Apellido>
        </Profesor>
      </Docentes>
      <Correlativas>
        <Corre>CS106B</Corre>
      </Correlativas>
    </Materia>
    <Materia Numero="CS124" Vacantes="60">
      <Titulo>From Languages to Information</Titulo>
      <Descripcion>Natural language processing. Cross-listed as LING180.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>Dan</Nombre>
          <Apellido>Jurafsky</Apellido>
        </Profesor>
      </Docentes>
      <Correlativas>
        <Corre>CS107</Corre>
        <Corre>CS109</Corre>
      </Correlativas>
    </Materia>
    <Materia Numero="CS143" Vacantes="90">
      <Titulo>Compilers</Titulo>
      <Descripcion>Principles and practices for design and implementation of compilers and interpreters.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>Alex</Nombre>
          <Segundo_Nombre>S.</Segundo_Nombre>
          <Apellido>Aiken</Apellido>
        </Profesor>
      </Docentes>
      <Correlativas>
        <Corre>CS107</Corre>
      </Correlativas>
    </Materia>
    <Materia Numero="CS145" Vacantes="130">
      <Titulo>Introduction to Databases</Titulo>
      <Descripcion>Database design and use of database management systems for applications.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>Jennifer</Nombre>
          <Apellido>Widom</Apellido>
        </Profesor>
      </Docentes>
      <Correlativas>
        <Corre>CS107</Corre>
      </Correlativas>
    </Materia>
    <Materia Numero="CS221" Vacantes="180">
      <Titulo>Artificial Intelligence: Principles and Techniques</Titulo>
      <Docentes>
        <Profesor>
          <Nombre>Andrew</Nombre>
          <Apellido>Ng</Apellido>
        </Profesor>
        <Profesor>
          <Nombre>Sebastian</Nombre>
          <Apellido>Thrun</Apellido>
        </Profesor>
      </Docentes>
    </Materia>
    <Materia Numero="CS228" Vacantes="110">
      <Titulo>Structured Probabilistic Models: Principles and Techniques</Titulo>
      <Descripcion>Using probabilistic modeling languages to represent complex domains.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>Daphne</Nombre>
          <Apellido>Koller</Apellido>
        </Profesor>
      </Docentes>
    </Materia>
    <Materia Numero="CS229" Vacantes="320">
      <Titulo>Machine Learning</Titulo>
      <Descripcion>A broad introduction to machine learning and statistical pattern recognition.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>Andrew</Nombre>
          <Apellido>Ng</Apellido>
        </Profesor>
      </Docentes>
    </Materia>
  </Departamento>
  <Departamento Codigo="EE">
    <Titulo>Electrical Engineering</Titulo>
    <Director>
      <Profesor>
        <Nombre>Mark</Nombre>
        <Segundo_Nombre>A.</Segundo_Nombre>
        <Apellido>Horowitz</Apellido>
      </Profesor>
    </Director>
    <Materia Numero="EE108A">
      <Titulo>Digital Systems I</Titulo>
      <Descripcion>Digital circuit, logic, and system design.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>Subhasish</Nombre>
          <Apellido>Mitra</Apellido>
        </Profesor>
      </Docentes>
    </Materia>
    <Materia Numero="EE108B">
      <Titulo>Digital Systems II</Titulo>
      <Descripcion>The design of processor-based digital systems.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>William</Nombre>
          <Segundo_Nombre>J.</Segundo_Nombre>
          <Apellido>Dally</Apellido>
        </Profesor>
        <Profesor>
          <Nombre>Oyekunle</Nombre>
          <Apellido>Olukotun</Apellido>
        </Profesor>
      </Docentes>
      <Correlativas>
        <Corre>EE108A</Corre>
        <Corre>CS106B</Corre>
      </Correlativas>
    </Materia>
  </Departamento>
  <Departamento Codigo="LING">
    <Titulo>Linguistics</Titulo>
    <Director>
      <Profesor>
        <Nombre>Beth</Nombre>
        <Apellido>Levin</Apellido>
      </Profesor>
    </Director>
    <Materia Numero="LING180" Vacantes="60">
      <Titulo>From Languages to Information</Titulo>
      <Descripcion>Natural language processing. Cross-listed as CS124.</Descripcion>
      <Docentes>
        <Profesor>
          <Nombre>Dan</Nombre>
          <Apellido>Jurafsky</Apellido>
        </Profesor>
      </Docentes>
      <Correlativas>
        <Corre>CS107</Corre>
        <Corre>CS109</Corre>
      </Correlativas>
    </Materia>
  </Departamento>
</Materias>
```

http://xpather.com/agWTW96O

In [ ]:
from lxml import etree

doc = etree.parse("materias.xml")

1. Encontrar todos los nodos Título (tanto de departamentos como de materias)
    - https://app.sli.do/event/ceybR8esJHrjVLtmkTu7ah/embed/polls/70010b84-9297-41cd-af23-424953093050

In [ ]:
print_xml(doc.xpath("/"))

2. Encontrar los apellidos de todos los directores de departamento
    - https://app.sli.do/event/ceybR8esJHrjVLtmkTu7ah/embed/polls/f5d9afff-f13f-4bf4-b489-44daf85dd6fb

In [ ]:
print_xml(doc.xpath("/"))

3. Listar todas las materias con más de 500 vacantes
    - https://app.sli.do/event/ceybR8esJHrjVLtmkTu7ah/embed/polls/7bd05e85-f248-430d-9002-f1a7c6ba9861

In [ ]:
print_xml(doc.xpath("/"))

4. Listar los nombres (Títulos) de departamentos que tienen una materia con correlativa "CS106B"
    - https://app.sli.do/event/jCGympGJS5AMQuvMJQbWnD/embed/polls/2b3cc078-54af-491a-b68a-0457b2770c8f

In [ ]:
print_xml(doc.xpath("/"))

5. Apellidos de todos los docentes que firman con segundo nombre
    - https://app.sli.do/event/jCGympGJS5AMQuvMJQbWnD/embed/polls/289e68d5-5c63-48df-8491-d2871098faf2

In [ ]:
print_xml(doc.xpath("/"))

6. Apellidos de todos los docentes que tienen materias con vacantes mayores o iguales a 100
    - https://app.sli.do/event/jCGympGJS5AMQuvMJQbWnD/embed/polls/28bd88c6-9ca2-45e9-b563-eb3346797af3

In [ ]:
print_xml(doc.xpath("/"))